<a href="https://www.kaggle.com/code/fabriciositto/top-10-countries-smokers-per-year-animated?scriptVersionId=143220656" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [ ]:
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [ ]:
df = pd.read_csv(r'/kaggle/input/us-smoking-trend/smoking.csv')

In [ ]:
df

# Preparing and checking the df

In [ ]:
df.describe()

In [ ]:
#change the names to easier code

for i in np.arange(df.columns.size):
    df.rename(columns= { df.columns[i] : df.columns[i].replace("Data",'').replace('.','').replace(' ','') } , inplace=True)
    
df.columns

In [ ]:
df.info() #no nulls

In [ ]:
df.duplicated().sum() # no duplicated

# All ready to graph

Para un año en especifico,
Vamos a intentar un grafico de barras donde:
*     en el centro esté el nombre del pais, 
*     a la izquierda un grafico de barras horizontales alineadas a la derecha representando la cantidad de hombres fumadores por pais
*     a la derecha un grafico de barras horizontales alineadas a la izquiera representando la cantidad de mujeres fumadoras por pais
*     animado con los años

![example](http://www.dataviz-inspiration.com/_next/image?url=%2F_next%2Fstatic%2Fmedia%2Fworld-mental-health-survey-2.28f9ab60.png&w=1920&q=75)

## Step by Step to create the final graph

In [ ]:
#create a example data
ex_data=df[df.Year==1980][['Country','SmokersFemale','SmokersMale']].iloc[1:10].reset_index(drop=True)
ex_data

In [ ]:
#the first single graph using express
fig = px.bar(ex_data,x='SmokersFemale',y='Country', range_x=(2500000,0) ,orientation='h', title='Female')

fig.layout.xaxis.title.text=''   
fig.layout.title.x=0.9 

fig.show()


In [ ]:
print(fig)

In [ ]:
#the fist single graph using go
data1=go.Bar(
    name='Female',
    x=ex_data['SmokersFemale'],
    y=ex_data['Country'],
    orientation='h',
)

layout1=go.Layout(
    title='Female',
    xaxis=dict(
        title='Smokers',
        range= [np.max(ex_data['SmokersFemale'])*1.1, 0]
    ),
    yaxis=dict(
        title='',
        color='#000'
    )
)

fig1= go.Figure(data=[data1],layout=layout1)
fig1.show()

In [ ]:
data2=go.Bar(
    name='Male',
    x=ex_data['SmokersMale'],
    y=ex_data['Country'],
    orientation='h',
)

layout2=go.Layout(
    title='Male',
    xaxis=dict(
        title='Smokers'
    ),
    yaxis=dict(
        title='',
        color='#000'
    )
)

fig2= go.Figure(data=[data2],layout=layout2)
fig2.show()

In [ ]:
fig_double= make_subplots( 
    rows=1, cols=2,
    column_widths=[0.4,0.4],
    specs=[[{'type':'bar'},{'type':'bar'}]]
    )

fig_double.add_trace(data1,row=1,col=1)


fig_double.add_trace(data2,row=1,col=2)

fig_double.layout.xaxis.range=[2500000,0]

fig_double.show()

In [ ]:
years=df.Year.unique()
years

In [ ]:
#starting with animation
fig_dict={
    'data':[],
    'layout':{},
    'frames':[]
};

#layout

#create the buttom
fig_dict['layout']['updatemenus'] = [
    {
        "buttons": [
            {
                "args": [None, {"frame": {"duration": 500, "redraw": False},
                                "fromcurrent": True, "transition": {"duration": 300,
                                                                    "easing": "quadratic-in-out"}}],
                "label": "Play",
                "method": "animate"
            },
            {
                "args": [[None], {"frame": {"duration": 0, "redraw": False},
                                  "mode": "immediate",
                                  "transition": {"duration": 0}}],
                "label": "Pause",
                "method": "animate"
            }
        ],
        "direction": "left",
        "pad": {"r": 10, "t": 87},
        "showactive": False,
        "type": "buttons",
        "x": 0.1,
        "xanchor": "right",
        "y": 0,
        "yanchor": "top"
    }
]

#DATA
year=years[0]
df_female=df[df.Year==year][['Country','SmokersFemale','SmokersMale']].sort_values(by='SmokersFemale',ascending=False).iloc[1:15].reset_index(drop=True)
data_female=go.Bar(
    name='Female',
    x=df_female['SmokersFemale'],
    y=df_female['Country'],
    orientation='h',
    text=df_female['Country']
)

fig_dict["data"].append(data_female)
    

#FRAMES

for year in years:
    frame = {"data": [], "name": str(year)}
    df_female_by_year=df[df.Year==year][['Country','SmokersFemale','SmokersMale']].sort_values(by='SmokersFemale',ascending=False).iloc[1:15].reset_index(drop=True)
    data_female=go.Bar(
        name='Female',
        x=df_female_by_year['SmokersFemale'],
        y=df_female_by_year['Country'],
        orientation='h',
        text=df_female_by_year['Country']
    )
    frame['data'].append(data_female)
    
    fig_dict['frames'].append(frame)

#create the figure
    
fig_animated = go.Figure(fig_dict)

fig_animated.show()

In [ ]:
print(fig_double)

In [ ]:
print(fig_animated)

## Full code to the final graph

In [ ]:
#start the structure

fig_double_animated={
    'data': [],
    'layout':(),
    'frames':[]
}

In [ ]:
#create a simplier array of dataframes with the focus info
years=df.Year.unique()    #all sorted years
dfs_by_years=[]
for year in years:
    df_by_year=df[df.Year==year][['Country','SmokersFemale','SmokersMale','SmokersTotal']].sort_values(by='SmokersTotal',ascending=False).iloc[1:10].reset_index(drop=True).sort_values(by='SmokersTotal',ascending=True)
    dfs_by_years.append(df_by_year)


In [ ]:
# create the data

data_fba=[
    go.Bar(
        name='Female',
        x=dfs_by_years[0]['SmokersFemale'],
        y=dfs_by_years[0]['Country'],
        orientation='h',
        text=dfs_by_years[0]['Country'],
        xaxis='x',
        yaxis='y',
        showlegend=True,
        textposition='outside'
    ),
    
    go.Bar(
        name='Male',
        x=dfs_by_years[0]['SmokersMale'],
        y=dfs_by_years[0]['Country'],
        orientation='h',
        text=dfs_by_years[0]['Country'],
        xaxis='x2',
        yaxis='y2',
        showlegend=True,
        textposition='outside'

    )
]


# add the data to the figure structure
fig_double_animated['data']=data_fba

In [ ]:
# create the layout


layout_fba={
    'title':{'text':'Top 10 countries with the most smokers per year (Female & Male)','x':0.5},
    'xaxis':{'anchor': 'y', 'domain': [0.0, 0.5], 'range': [ df[df['Country']!='World']['SmokersMale'].max()*1.02 , 0 ]},
    'xaxis2': {'anchor': 'y2', 'domain': [0.5, 1.0], 'range': [ 0 , df[df['Country']!='World']['SmokersMale'].max()*1.02 ]},
    'yaxis': {'anchor': 'x', 'domain': [0.0, 1.0], 'showticklabels': False},
    'yaxis2': {'anchor': 'x2', 'domain': [0.0, 1.0], 'showticklabels': False},
    'height':700
}


#add the layout to the figure structure
fig_double_animated['layout']=layout_fba

In [ ]:
#little progress check

fig_test=go.Figure(fig_double_animated)
fig_test.show()

In [ ]:
# create the frame

frames_fba=[]
slider_steps=[]

for i in np.arange(len(years)):
    frame={'data':[],'name':str(years[i])}
    data_female=go.Bar(
        name='Female',
        x=dfs_by_years[i]['SmokersFemale'],
        y=dfs_by_years[i]['Country'],
        orientation='h',
        text=dfs_by_years[i]['Country'],
        xaxis='x',
        yaxis='y',
        textposition='outside'

    )
    data_male=go.Bar(
        name='Male',
        x=dfs_by_years[i]['SmokersMale'],
        y=dfs_by_years[i]['Country'],
        orientation='h',
        text=dfs_by_years[i]['Country'],
        xaxis='x2',
        yaxis='y2',
        textposition='outside'
    )
    
    frame['data'].append(data_female)
    frame['data'].append(data_male)
    
    frames_fba.append(frame)
    #slider's step
    step={"args": [ [years[i]] , { "frame" : {"duration": 500 , "redraw": False},
         "mode": "immediate",
         "transition": {"duration": 500}}],  
        "label": str(years[i]),
        "method": "animate"
    }
    
    slider_steps.append(step)


    
    
#boton and slider

boton=[
    {
        "buttons": [
            {
                "args": [None, {"frame": {"duration": 500, "redraw": False},
                                "fromcurrent": True, "transition": {"duration": 500,
                                                                    "easing": "quadratic-in-out"}}],
                "label": "Play",
                "method": "animate"
            },
            {
                "args": [[None], {"frame": {"duration": 0, "redraw": False},
                                  "mode": "immediate",
                                  "transition": {"duration": 0}}],
                "label": "Pause",
                "method": "animate"
            }
        ],
        "direction": "left",
        "pad": {"r": 10, "t": 87},
        "showactive": False,
        "type": "buttons",
        "x": 0.1,
        "xanchor": "right",
        "y": 0,
        "yanchor": "top"
    }
]

sliders = {
    "active": 0,
    "yanchor": "top",
    "xanchor": "left",
    "currentvalue": {
        "font": {"size": 20},
        "prefix": "Year:",
        "visible": True,
        "xanchor": "right"
    },
    "transition": {"duration": 500, "easing": "cubic-in-out"},
    "pad": {"b": 10, "t": 50},
    "len": 0.9,
    "x": 0.1,
    "y": 0,
    "steps": slider_steps
}

# integrate all
fig_double_animated['layout']['updatemenus']= boton
fig_double_animated["layout"]["sliders"] = [sliders]

fig_double_animated['frames']=frames_fba

In [ ]:
fig_animated=go.Figure(fig_double_animated)
fig_animated.show()

In [ ]:
#NOW USING THE PERCENTAGE
# (ALL IN ONE CODE CELL)

fig_double_animated2={
    'data': [],
    'layout':(),
    'frames':[]
}

######################################################################
#create a simplier array of dataframes with the focus info
years2=df.Year.unique()    #all sorted years
dfs_by_years2=[]
for year in years2:
    df_by_year2=df[df.Year==year][['Country','PercentageFemale','PercentageMale','PercentageTotal']].sort_values(by='PercentageTotal',ascending=False).iloc[1:10].reset_index(drop=True).sort_values(by='PercentageTotal',ascending=True)
    dfs_by_years2.append(df_by_year2)
    
#######################################################################
# create the data

data_fba2=[
    go.Bar(
        name='Female',
        x=dfs_by_years2[0]['PercentageFemale'],
        y=dfs_by_years2[0]['Country'],
        orientation='h',
        text=dfs_by_years2[0]['Country'],
        xaxis='x',
        yaxis='y',
        showlegend=True,
        textposition='outside'
    ),
    
    go.Bar(
        name='Male',
        x=dfs_by_years2[0]['PercentageMale'],
        y=dfs_by_years2[0]['Country'],
        orientation='h',
        text=dfs_by_years[0]['Country'],
        xaxis='x2',
        yaxis='y2',
        showlegend=True,
        textposition='outside'

    )
]


# add the data to the figure structure
fig_double_animated2['data']=data_fba2


#####################################################################333


# create the layout
layout_fba2={
    'title':{'text':'Top 10 countries with the most percentage of smokers per year (Female & Male)','x':0.5},
    'xaxis':{'anchor': 'y', 'domain': [0.0, 0.5], 'range': [ df[df['Country']!='World'][['PercentageMale','PercentageFemale']].max().max()*1.02 , 0 ]},
    'xaxis2': {'anchor': 'y2', 'domain': [0.5, 1.0], 'range': [ 0 , df[df['Country']!='World'][['PercentageMale','PercentageFemale']].max().max()*1.02 ]},
    'yaxis': {'anchor': 'x', 'domain': [0.0, 1.0], 'showticklabels': False},
    'yaxis2': {'anchor': 'x2', 'domain': [0.0, 1.0], 'showticklabels': False},
    'height':500
}

#add the layout to the figure structure
fig_double_animated2['layout']=layout_fba2


#########################################################################


# create the frame
frames_fba2=[]
slider_steps2=[]

for i in np.arange(len(years2)):
    frame={'data':[],'name':str(years2[i])}
    data_female2=go.Bar(
        name='Female',
        x=dfs_by_years2[i]['PercentageFemale'],
        y=dfs_by_years2[i]['Country'],
        orientation='h',
        text=dfs_by_years2[i]['Country'],
        xaxis='x',
        yaxis='y',
        textposition='outside'

    )
    data_male2=go.Bar(
        name='Male',
        x=dfs_by_years2[i]['PercentageMale'],
        y=dfs_by_years2[i]['Country'],
        orientation='h',
        text=dfs_by_years2[i]['Country'],
        xaxis='x2',
        yaxis='y2',
        textposition='outside'
    )
    
    frame['data'].append(data_female2)
    frame['data'].append(data_male2)
    
    frames_fba2.append(frame)
    #slider's step
    step={"args": [ [years2[i]] , { "frame" : {"duration": 500 , "redraw": False},
         "mode": "immediate",
         "transition": {"duration": 500}}],  
        "label": str(years2[i]),
        "method": "animate"
    }
    
    slider_steps2.append(step)


#boton and slider

boton2=[
    {
        "buttons": [
            {
                "args": [None, {"frame": {"duration": 500, "redraw": False},
                                "fromcurrent": True, "transition": {"duration": 500,
                                                                    "easing": "quadratic-in-out"}}],
                "label": "Play",
                "method": "animate"
            },
            {
                "args": [[None], {"frame": {"duration": 0, "redraw": False},
                                  "mode": "immediate",
                                  "transition": {"duration": 0}}],
                "label": "Pause",
                "method": "animate"
            }
        ],
        "direction": "left",
        "pad": {"r": 10, "t": 87},
        "showactive": False,
        "type": "buttons",
        "x": 0.1,
        "xanchor": "right",
        "y": 0,
        "yanchor": "top"
    }
]

sliders2 = {
    "active": 0,
    "yanchor": "top",
    "xanchor": "left",
    "currentvalue": {
        "font": {"size": 20},
        "prefix": "Year:",
        "visible": True,
        "xanchor": "right"
    },
    "transition": {"duration": 500, "easing": "cubic-in-out"},
    "pad": {"b": 10, "t": 50},
    "len": 0.9,
    "x": 0.1,
    "y": 0,
    "steps": slider_steps
}

# integrate all
fig_double_animated2['layout']['updatemenus']= boton2
fig_double_animated2["layout"]["sliders"] = [sliders2]

fig_double_animated2['frames']=frames_fba2


fig_animated2=go.Figure(fig_double_animated2)
fig_animated2.show()

In [ ]:
df[df['Country']!='World'][['PercentageMale','PercentageFemale']].max().max()

### Future upgrades

* colors by continent
* put the countries' names on the middle of the graph keeping the animation